In [ ]:
pip install sodapy

In [ ]:
pip install google-cloud-bigquery

Extraction of the 311 Service Request Dataset

In [ ]:
import pandas as pd
from sodapy import Socrata

data_url='data.cityofnewyork.us'    # The Host Name for the API endpoint (the https:// part will be added automatically)
data_set='erm2-nwe9'    # The data set at the API endpoint (311 data in this case)
app_token='fRtgN6KKZ5bEIkbEB12tbzK7T'   # The app token created in the prior steps
client = Socrata(data_url,app_token)      # Create the client to point to the API endpoint
# Set the timeout to 60 seconds
client.timeout = 60

In [ ]:
metadata = client.get_metadata(data_set)
[x['name'] for x in metadata['columns']]
['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name', 'Complaint Type', 'Descriptor',
'Location Type', 'Incident Zip', 'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
'Intersection Street 1', 'Intersection Street 2', 'Address Type', 'City', 'Landmark', 'Facility Type',
'Status', 'Due Date', 'Resolution Description', 'Resolution Action Updated Date', 'Community Board', 'BBL',
'Borough', 'X Coordinate (State Plane)', 'Y Coordinate (State Plane)', 'Open Data Channel Type',
'Park Facility Name', 'Park Borough', 'Vehicle Type', 'Taxi Company Borough', 'Taxi Pick Up Location',
'Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment', 'Latitude',
'Longitude', 'Location', 'Zip Codes', 'Community Districts', 'Borough Boundaries', 'City Council Districts',
'Police Precincts']

['Unique Key',
 'Created Date',
 'Closed Date',
 'Agency',
 'Agency Name',
 'Complaint Type',
 'Descriptor',
 'Location Type',
 'Incident Zip',
 'Incident Address',
 'Street Name',
 'Cross Street 1',
 'Cross Street 2',
 'Intersection Street 1',
 'Intersection Street 2',
 'Address Type',
 'City',
 'Landmark',
 'Facility Type',
 'Status',
 'Due Date',
 'Resolution Description',
 'Resolution Action Updated Date',
 'Community Board',
 'BBL',
 'Borough',
 'X Coordinate (State Plane)',
 'Y Coordinate (State Plane)',
 'Open Data Channel Type',
 'Park Facility Name',
 'Park Borough',
 'Vehicle Type',
 'Taxi Company Borough',
 'Taxi Pick Up Location',
 'Bridge Highway Name',
 'Bridge Highway Direction',
 'Road Ramp',
 'Bridge Highway Segment',
 'Latitude',
 'Longitude',
 'Location',
 'Zip Codes',
 'Community Districts',
 'Borough Boundaries',
 'City Council Districts',
 'Police Precincts']

In [ ]:
# Retrieve the first 2000 results returned as JSON object from the API where the range falls within the years
where_clause = "complaint_type = 'Traffic Signal Condition' AND created_date BETWEEN '2018-01-01' AND '2023-12-31'"
results = client.get(data_set, where=where_clause,limit=100000000)

In [ ]:
# Convert the list of dictionaries to a Pandas data frame
df = pd.DataFrame.from_records(results)
# Save the data frame to a CSV file
df.to_csv("my_311_data.csv")
row = 1
df.at[row, 'incident_zip'] = 11234

In [ ]:
df['created_date'].sort_values() #just checking to see if all the dates are valid

225138    2018-01-01T00:27:00.000
225137    2018-01-01T01:08:00.000
225136    2018-01-01T01:12:00.000
225135    2018-01-01T02:24:00.000
225134    2018-01-01T03:23:00.000
                   ...           
4         2023-11-24T23:15:00.000
3         2023-11-24T23:28:00.000
2         2023-11-24T23:37:00.000
1         2023-11-25T00:25:00.000
0         2023-11-25T00:28:00.000
Name: created_date, Length: 225139, dtype: object

In [ ]:
# Assuming df is your DataFrame
df['unique_key'] = pd.to_numeric(df['unique_key'], errors='coerce')

# Replace NaN values with a default integer value or any other appropriate handling
default_value = 0  # Replace with your desired default value
df['unique_key'].fillna(default_value, inplace=True)

# Convert the column to integers
df['unique_key'] = df['unique_key'].astype(int)


df['incident_zip'] = pd.to_numeric(df['incident_zip'], errors='coerce')
df['incident_zip'].fillna(default_value, inplace=True)
df['incident_zip'] = df['incident_zip'].astype(int)


df['x_coordinate_state_plane'] = pd.to_numeric(df['x_coordinate_state_plane'], errors='coerce')
df['x_coordinate_state_plane'].fillna(default_value, inplace=True)
df['x_coordinate_state_plane'] = df['x_coordinate_state_plane'].astype(int)

df['y_coordinate_state_plane'] = pd.to_numeric(df['y_coordinate_state_plane'], errors='coerce')
df['y_coordinate_state_plane'].fillna(default_value, inplace=True)
df['y_coordinate_state_plane'] = df['y_coordinate_state_plane'].astype(int)

df['bbl'] = pd.to_numeric(df['bbl'], errors='coerce')
df['bbl'].fillna(default_value, inplace=True)
df['bbl'] = df['bbl'].astype(int)


uploading to BQ


In [ ]:

#from collab to google cloud storage
from google.cloud import storage
import pandas as pd

# Assuming df is your DataFrame
bucket_name = 'your_bucket_name'  # Replace with your GCS bucket name
file_name = 'your_file_name.csv'  # Replace with your desired file name

# Save DataFrame to CSV
df.to_csv(file_name, index=False)

# Upload the CSV file to GCS
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_name)
blob.upload_from_filename(file_name)

# from google storage to bq
from google.cloud import bigquery

dataset_id = 'your_dataset_id'  # Replace with your BigQuery dataset ID
table_id = 'your_table_id'  # Replace with your BigQuery table ID

# Create BigQuery dataset if it doesn't exist
client = bigquery.Client()
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
client.create_dataset(dataset, exists_ok=True)

# Create BigQuery table
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig(schema=schema)

# Load data from GCS to BigQuery
uri = f"gs://{bucket_name}/{file_name}"
load_job = client.load_table_from_uri(uri, table_ref, job_config=job_config)
load_job.result()  # Wait for the job to complete

print(f"Loaded {load_job.output_rows} rows into {dataset_id}.{table_id}")



In [ ]:
import datetime

from google.cloud import bigquery
import pandas
import pytz

# Construct a BigQuery client object.
client = bigquery.Client()

table_id = 'cis4400project-403800.projectDatasets.311_service_requests_backup'

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField("unique_key", bigquery.enums.SqlTypeNames.INTEGER,mode="NULLABLE"),
        # Indexes are written if included in the schema by name.
        bigquery.SchemaField("created_date", bigquery.enums.SqlTypeNames.TIMESTAMP,mode="NULLABLE"),
        bigquery.SchemaField("agency", bigquery.enums.SqlTypeNames.STRING, mode="NULLABLE"),
        bigquery.SchemaField("agency_name", bigquery.enums.SqlTypeNames.STRING, mode="NULLABLE"),
        bigquery.SchemaField("complaint_type", bigquery.enums.SqlTypeNames.STRING, mode="NULLABLE"),
        bigquery.SchemaField("descriptor", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("incident_zip", bigquery.enums.SqlTypeNames.INTEGER,mode="NULLABLE"),
        bigquery.SchemaField("intersection_street_1", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("intersection_street_2", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("address_type", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("city", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("facility_type", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("status", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("community_board", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("borough", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("x_coordinate_state_plane", bigquery.enums.SqlTypeNames.INTEGER,mode="NULLABLE"),
        bigquery.SchemaField("y_coordinate_state_plane", bigquery.enums.SqlTypeNames.INTEGER,mode="NULLABLE"),
        bigquery.SchemaField("open_data_channel_type", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("park_facility_name", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("park_borough", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("latitude", bigquery.enums.SqlTypeNames.FLOAT,mode="NULLABLE"),
        bigquery.SchemaField("longitude", bigquery.enums.SqlTypeNames.FLOAT,mode="NULLABLE"),
        bigquery.SchemaField("location", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("closed_date", bigquery.enums.SqlTypeNames.TIMESTAMP,mode="NULLABLE"),
        bigquery.SchemaField("resolution_description", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("resolution_action_updated_date", bigquery.enums.SqlTypeNames.TIMESTAMP,mode="NULLABLE"),
        bigquery.SchemaField("incident_address", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("street_name", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("cross_street_1", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("cross_street_2", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE"),
        bigquery.SchemaField("bbl", bigquery.enums.SqlTypeNames.INTEGER,mode="NULLABLE"),
        bigquery.SchemaField("due_date", bigquery.enums.SqlTypeNames.STRING,mode="NULLABLE")
    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

ArrowTypeError: ignored

In [ ]:
df.head(5)

,unique_key,created_date,agency,agency_name,complaint_type,descriptor,incident_zip,intersection_street_1,intersection_street_2,address_type,...,location,closed_date,resolution_description,resolution_action_updated_date,incident_address,street_name,cross_street_1,cross_street_2,bbl,due_date
0,59540294,2023-11-25T00:28:00.000,DOT,Department of Transportation,Traffic Signal Condition,Controller,11385,CENTRAL AVENUE,67 PLACE,INTERSECTION,...,"{'latitude': '40.703379417547566', 'longitude'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59534495,2023-11-25T00:25:00.000,DOT,Department of Transportation,Traffic Signal Condition,Controller,11234,FLATBUSH AVE,TOYS R US,INTERSECTION,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59535482,2023-11-24T23:37:00.000,DOT,Department of Transportation,Traffic Signal Condition,Controller,11234,FLATBUSH AVENUE,AVENUE V,INTERSECTION,...,"{'latitude': '40.60820274118944', 'longitude':...",2023-11-24T23:55:00.000,Service Request status for this request is ava...,2023-11-24T23:55:00.000,NaN,NaN,NaN,NaN,NaN,NaN
3,59540095,2023-11-24T23:28:00.000,DOT,Department of Transportation,Traffic Signal Condition,Post,11234,FLATBUSH AVENUE,AVENUE V,INTERSECTION,...,"{'latitude': '40.60820274118944', 'longitude':...",2023-11-24T23:55:00.000,Service Request status for this request is ava...,2023-11-24T23:55:00.000,NaN,NaN,NaN,NaN,NaN,NaN
4,59535486,2023-11-24T23:15:00.000,DOT,Department of Transportation,Traffic Signal Condition,Controller,11207,PENNSYLVANIA AVENUE,STANLEY AVENUE,INTERSECTION,...,"{'latitude': '40.656979266383345', 'longitude'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Extractions For Collisions dataset

In [ ]:
data_url2='data.cityofnewyork.us'    # The Host Name for the API endpoint (the https:// part will be added automatically)
data_set2='h9gi-nx95'    # The data set at the API endpoint (311 data in this case)
app_token2='fRtgN6KKZ5bEIkbEB12tbzK7T'   # The app token created in the prior steps
client = Socrata(data_url2,app_token2)      # Create the client to point to the API endpoint
# Set the timeout to 60 seconds
client.timeout = 60
# Retrieve the first 2000 results returned as JSON object from the API
# The SoDaPy library converts this JSON object to a Python list of dictionaries
where_clause2 = "date_extract_y(crash_date) BETWEEN 2018 AND 2023"
results2 = client.get(data_set2, where=where_clause2, limit=100000000)

In [ ]:
# Convert the list of dictionaries to a Pandas data frame
df2 = pd.DataFrame.from_records(results2)
# Save the data frame to a CSV file
df2.to_csv("collisions.csv")

In [ ]:
df2['crash_date'].sort_values()

730557    2018-01-01T00:00:00.000
730049    2018-01-01T00:00:00.000
730064    2018-01-01T00:00:00.000
730069    2018-01-01T00:00:00.000
724687    2018-01-01T00:00:00.000
                   ...           
843680    2023-11-21T00:00:00.000
843681    2023-11-21T00:00:00.000
843682    2023-11-21T00:00:00.000
843669    2023-11-21T00:00:00.000
843301    2023-11-21T00:00:00.000
Name: crash_date, Length: 856052, dtype: object

Uploading COLLISIONS to BQ


In [ ]:
import datetime

from google.cloud import bigquery
import pandas
import pytz

# Construct a BigQuery client object.
client = bigquery.Client()

table_id = 'cis4400project-403800.projectDatasets.vehicle_collisions'

job_config = bigquery.LoadJobConfig(
    # Specify a (partial) schema. All columns are always written to the
    # table. The schema is used to assist in data type definitions.
    schema=[
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        # Indexes are written if included in the schema by name.
        bigquery.SchemaField("crash_date", bigquery.enums.SqlTypeNames.TIMESTAMP),
        bigquery.SchemaField("crash_time", bigquery.enums.SqlTypeNames.TIMESTAMP),
        bigquery.SchemaField("on_street_name", bigquery.enums.SqlTypeNames.STRING, mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE"),
        bigquery.SchemaField("", bigquery.enums.SqlTypeNames., mode="NULLABLE")

    ],
    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Fact Tables ETL


Notes:

Have not defined the path yet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ETL Complaint Facts
# If using the native Google BigQuery API module:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
# import credentials
import pandas as pd
import os
import pyarrow
from datetime import datetime
from google.oauth2 import service_account

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.DataFrame
# Set the name of the dimension
fact_name = 'complaints'

# Set the GCP Project, dataset and table name
gcp_project = 'cis4400project-403800'
bq_dataset = '311_service_requests'
table_name = fact_name + '_fact'
# Construct the full BigQuery path to the table
fact_table_path = ".".join([gcp_project,bq_dataset,table_name])

# Set the path to the source data files
# For Linux use something like    /home/username/python_etl
# For Mac use something like     /users/username/python_etl
# file_source_path = 'c:\\Python_ETL'
file_source_path = '/content' #what do we want to define here
path_to_service_account_key_file = '/content/drive/MyDrive/cis4400project-403800-81bc1b0a588c(1).json'
def transform_data( df):
    """
    transform_data
    Accepts a data frame
    Performs any specific cleaning and transformation steps on the dataframe
    Returns the modified dataframe
    """
    # Convert the date_of_birth to a datetime64 data type. 2012-08-21 04:12:16.827
    df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], format='%m/%d/%Y')
    # Convert the postal code into a string
    df['incident_zip'] =  df['incident_zip'].astype(str)
    return df
def upload_bigquery_table(bqclient, table_path, write_disposition, df):
    """
    upload_bigquery_table
    Accepts a path to a BigQuery table, the write disposition and a dataframe
    Loads the data into the BigQuery table from the dataframe.
    for credentials.
    The write disposition is either
    write_disposition="WRITE_TRUNCATE"  Erase the target data and load all new data.
    write_disposition="WRITE_APPEND"    Append to the existing table
    """
    try:

        job_config = bigquery.LoadJobConfig(write_disposition=write_disposition)

        # Submit the job
        job = bqclient.load_table_from_dataframe(df, table_path, job_config=job_config)

        # Show the job results
        job.result()
    except Exception as err:
        print("Failed to load BigQuery Table.", err)
        # os._exit(-1)


def bigquery_table_exists(table_path, bqclient):
    """
    bigquery_table_exists
    Accepts a path to a BigQuery table
    Checks if the BigQuery table exists.
    Returns True or False
    """
    try:
        bqclient.get_table(table_path)  # Make an API request.
        return True
    except NotFound:
        # print("Table {} is not found.".format(table_id))
        return False
def query_bigquery_table(table_path, bqclient, surrogate_key):
    """
    query_bigquery_table
    Accepts a path to a BigQuery table and the name of the surrogate key
    Queries the BigQuery table but leaves out the update_timestamp and surrogate key columns
    Returns the dataframe
    """
    bq_df = pd.DataFrame
    sql_query = 'SELECT * EXCEPT ( update_timestamp, '+surrogate_key+') FROM `' + table_path + '`'
    try:
        bq_df = bqclient.query(sql_query).to_dataframe()
    except Exception as err:
        print("error")
    return bq_df

def dimension_lookup( dimension_name='agency', lookup_columns=['agency', 'agency_name'], df=df):
    """
    dimension_lookup
    Lookup the lookup_columns in the dimension_name and return the associated surrogate keys
    Returns dataframe augmented with the surrogate keys
    """
    bq_df = pd.DataFrame
    surrogate_key = dimension_name+"_dim_id"
    dimension_table_path = ".".join([gcp_project,bq_dataset,dimension_name+"_dimension"])
    # Fetch the existing table
    bq_df = query_bigquery_table( dimension_table_path, bqclient, surrogate_key)
    #print(bq_df)
    # Melt the dimension dataframe into an index with the lookup columns
    m = bq_df.melt(id_vars=lookup_columns, value_vars=surrogate_key)
    #print(m)
    # Rename the "value" column to the surrogate key column name
    m=m.rename(columns={"value":surrogate_key})
    # Merge with the fact table record
    df = df.merge(m, on=lookup_columns, how='left')
    # Drop the "variable" column and the lookup columns
    df = df.drop(columns=lookup_columns)
    df = df.drop(columns="variable")
    #print(df)
    return df

def date_dimension_lookup(dimension_name='date', lookup_column='created_date', df=df):
    """
    date_dimension_lookup
    Lookup the lookup_columns in a date dimension and return the associated surrogate keys
    Returns dataframe augmented with the surrogate keys
    """
    bq_df = pd.DataFrame
    surrogate_key = dimension_name+"_dim_id"
    dimension_table_path = ".".join([gcp_project,bq_dataset,dimension_name+"_dimension"])
    # Fetch the existing table
    bq_df = query_bigquery_table( dimension_table_path, bqclient, surrogate_key)
    bq_df["full_date"] = pd.to_datetime(bq_df.full_date, format="%Y-%m-%d %H:%M:%S")
    # Return just the date portion
    bq_df["full_date"] = bq_df.full_date.dt.date

    # Dates in the 311 data look like this: 2017-08-11T11:57:00.000
    # Extract the date from 'created_date' column
    df[lookup_column] = pd.to_datetime(df[lookup_column], format="%Y-%m-%d %H:%M:%S.%f")
    # Return just the date portion
    df[lookup_column] = df[lookup_column].dt.date

    # Melt the dimension dataframe into an index with the lookup columns
    m = bq_df.melt(id_vars='full_date', value_vars=surrogate_key)
    # Rename the "value" column to the surrogate key column name
    m=m.rename(columns={"value":lookup_column+"_dim_id"})

    # Merge with the fact table record on the created_date
    df = df.merge(m, left_on=lookup_column, right_on='full_date', how='left')

    # Drop the "variable" column and the lookup columns
    df = df.drop(columns=lookup_column)
    df = df.drop(columns="variable")
    df = df.drop(columns="full_date")
    return df



def time_dimension_lookup( dimension_name='time', lookup_column='created_date', df=df):
    """
    time_dimension_lookup
    Lookup the lookup_columns in the time dimension and return the associated surrogate key
    Returns dataframe augmented with the surrogate keys
    """
    bq_df = pd.DataFrame
    surrogate_key = dimension_name+"_dim_id"
    dimension_table_path = ".".join([gcp_project,bq_dataset,dimension_name+"_dimension"])

    # Dates in the 311 data look like this: 2017-08-11T11:57:00.000
    # We can strip off the time portion after the letter "T" to ge the hours and minutes
    # time_dim_id = (hours*60)+minutes+1
    # Example:  Time is 22:07  so  (22*60)+7+1 = 1328
    # Extract the date from 'created_date' column and save it in a temporary column
    df[lookup_column+"_newdate"] = pd.to_datetime(df[lookup_column], format="%Y-%m-%d %H:%M:%S.%f")
    # Strip off the hours and minutes portions
    df[lookup_column+"_hours"] = df[lookup_column+"_newdate"].dt.strftime("%H").astype(int)
    # df[lookup_column+"_hours"] = df[lookup_column+"_hours"].astype(int)
    df[lookup_column+"_minutes"] = df[lookup_column+"_newdate"].dt.strftime("%M").astype(int)
    # df[lookup_column+"_minutes"] = df[lookup_column+"_minutes"].astype(int)
    # Now assign the time_dim_id
    df[surrogate_key] = (df[lookup_column+"_hours"]*60)+df[lookup_column+"_minutes"]+1
    print("Surrogate key is: ", surrogate_key)
    print(df[surrogate_key])
    # Drop the lookup time columns
    df = df.drop(columns=lookup_column+"_newdate")
    df = df.drop(columns=lookup_column+"_hours")
    df = df.drop(columns=lookup_column+"_minutes")
    return df

In [ ]:
if __name__ == "__main__":
    df = pd.DataFrame
    # Create the BigQuery Client
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_to_service_account_key_file

    # Construct a BigQuery client object
    bqclient = bigquery.Client()

    # Load in the data file
    # Load in the data file
    with open(file_source_path, 'r') as data:
            df = pd.read_csv(data)
        # Set all of the column names to lower case letters
    #print(df.head())
    df = df.rename(columns=str.lower)    # If city is empty, fill it in with NEW YORK
    df.city = df.city.fillna('NEW YORK')

    # Consider removing columns that we will never use  df.drop([....])

    # Lookup the agency dimension record  agency_dim_id
    df = dimension_lookup( dimension_name='agency', lookup_columns=['agency', 'agency_name'], df=df)

    # Lookup the location dimension record  location_dim_id
    #df = dimension_lookup( dimension_name='location', lookup_columns=['borough', 'city', 'incident_address', 'incident_zip', 'latitude', 'longitude'], df=df)

    # Lookup the channel  dimension record  channel_dim_id
    #df = dimension_lookup( dimension_name='channel', lookup_columns=['open_data_channel_type', 'status'], df=df)

    # Lookup the complaint_type  dimension record  complaint_type_dim_id
    #df = dimension_lookup dimension_name='complaint_type', lookup_columns=['complaint_type', 'descriptor'], df=df)

    # Lookup the time dimension record using the time part of the created_date
    # Note - do this before looking up the date dimension
    #df = time_dimension_lookup( dimension_name='time', lookup_column='created_date', df=df)
    # The time_dimension_lookup returns a column named 'time_dim_id'. Rename this to the 'created_time_dim_id'
    #df = df.rename(columns={'time_dim_id' : 'created_time_dim_id'})

    # Lookup the created_date dimension record
   # df = date_dimension_lookup(dimension_name='date', lookup_column='created_date', df=df)

    # Lookup the closed_date dimension record
    #df = date_dimension_lookup(dimension_name='date', lookup_column='closed_date', df=df)

    # A list of all of the surrogate keys
    # For transaction grain, also include the 'unique_key' column
    surrogate_keys=['agency_dim_id','complaint_type_dim_id','channel_dim_id','location_dim_id','created_date_dim_id','created_time_dim_id','closed_date_dim_id']

    # Remove all of the other non-surrogate key columns
    df = df[surrogate_keys]

    # For daily snapshot grain we:
    # 1) Add a 'complaint_count' fact
    # 2) Use Group By to count up the number of complaints, per location, per agency, etc. per day
    # For transaction grain add in the unique_key but skip the above two steps.

    # Add a complaint count (for daily snapshot grain)
    df['complaint_count'] = 1
    # Count up the number of complaints per agency, per location, etc. per day
    #df = df.groupby(surrogate_keys)['complaint_count'].agg('count').reset_index()

    # See if the target table exists
    target_table_exists = bigquery_table_exists(fact_table_path, bqclient )
    # If the target table does not exist, load all of the data into a new table
    if not target_table_exists:
        build_new_table( bqclient, fact_table_path, df)
    # If the target table exists, then perform an incremental load
    if target_table_exists:
        insert_existing_table( bqclient, fact_table_path, df)

IsADirectoryError: ignored

311 Dimensions

In [ ]:
# If using the native Google BigQuery API module:
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import os
import pyarrow
import logging
from datetime import datetime
import credentials
from google.oauth2 import service_account

# If using a service account key file, save the path to that file in credentials.py and import credentials
path_to_service_account_key_file = "/content/cis4400project-403800-81bc1b0a588c(1).json"
#!pip install credentials

# Set the name of the dimension
dimension_name = 'agency'

# Set the name of the surrogate key
surrogate_key = f"{dimension_name}_dim_id"

# Set the name of the business key
business_key = f'{dimension_name}_id'

# Set the GCP Project, dataset and table name
gcp_project = 'cis4400project-403800 '
bq_dataset = '311_service_requests'
table_name = f"{dimension_name}_dimension"
# Construct the full BigQuery path to the table
dimension_table_path = f"{gcp_project}.{bq_dataset}.{table_name}"

# Set the path to the source data files. Use double-slash for Windows paths C:\\myfolder
# For Linux use forward slashes    /home/username/python_etl
# For Mac use forward slashes      /users/username/python_etl
# file_source_path = 'c:\\Python_ETL'
# file_source_path = 'C:\\Users\\rholo\\OneDrive\\Documents\\classes\\4400\\311'
file_source_path = '/Contnet' #also have not defined this yet

def transform_data(df: pd.DataFrame):
    """
    transform_data
    Accepts a data frame
    Performs any specific cleaning and transformation steps on the dataframe
    Returns the modified dataframe
    This function can be modified based on required changes
    """
    # Select the columns for this dimension
    column_list = ['agency','agency_name']
    df = df[column_list]
    # Remove duplicates
    df = df.drop_duplicates()
    return df

def create_bigquery_client():
    """
    create_bigquery_client
    Creates a BigQuery client using the path to the service account key file
    for credentials.
    Returns the BigQuery client object
    """
    try:
        # If authenticating using a service account key file, use the following code:
        # bqclient = bigquery.Client.from_service_account_json(credentials.path_to_service_account_key_file)
        # Google Colab authentication already completed
        bqclient = bigquery.Client(gcp_project)
        return bqclient
    except Exception as err:
        print("error")
        # os._exit(-1)
    return bqclient

def upload_bigquery_table(bqclient, table_path, write_disposition, df):
    """
    upload_bigquery_table
    Accepts a path to a BigQuery table, the write disposition and a dataframe
    Loads the data into the BigQuery table from the dataframe.
    for credentials.
    The write disposition is either
    write_disposition="WRITE_TRUNCATE"  Erase the target data and load all new data.
    write_disposition="WRITE_APPEND"    Append to the existing table
    """
    try:
        # Set up a BigQuery job configuration with the write_disposition.
        job_config = bigquery.LoadJobConfig(write_disposition=write_disposition)

        # Submit the job
        print(type(bqclient))
        job = bqclient.load_table_from_dataframe(df, table_path, job_config=job_config)
        # Show the job results
    except Exception as err:
        print(err)
        #os._exit(-1)

def bigquery_table_exists(bqclient, table_path):
    """
    bigquery_table_exists
    Accepts a path to a BigQuery table
    Checks if the BigQuery table exists.
    Returns True or False
    """
    try:
        bqclient.get_table(table_path)  # Make an API request.
        return True
    except NotFound:
        return False

def query_bigquery_table(table_path, bqclient, surrogate_key):
    """
    query_bigquery_table
    Accepts a path to a BigQuery table and the name of the surrogate key
    Queries the BigQuery table but leaves out the update_timestamp and surrogate key columns
    Returns the dataframe
    """
    bq_df = pd.DataFrame
    sql_query = 'SELECT * EXCEPT ( update_timestamp, '+surrogate_key+') FROM `' + table_path + '`'
    try:
        bq_df = bqclient.query(sql_query).to_dataframe()
    except Exception as err:
        print("error")
    return bq_df

def add_surrogate_key(df, dimension_name='customers', offset=1):
    """
    add_surrogate_key
    Accepts a data frame and inserts an integer identifier as the first column
    Returns the modified dataframe
    """
    # Reset the index to count from 0
    df.reset_index(drop=True, inplace=True)
    # Add the new surrogate key starting from offset
    df.insert(0, dimension_name+'_dim_id', df.index+offset)
    return df

def build_new_table(bqclient, dimension_table_path, dimension_name, df):
    """
    build_new_table
    Accepts a path to a dimensional table, the dimension name and a data frame
    Add the surrogate key and a record timestamp to the data frame
    Inserts the contents of the dataframe to the dimensional table.
    """
    # Add a surrogate key
    df = add_surrogate_key(df, dimension_name, 1)
    # Add the update timestamp
    # Upload the dataframe to the BigQuery table
    upload_bigquery_table(bqclient, dimension_table_path, "WRITE_TRUNCATE", df)

# Program main
# Load the CSV File into a dataframe
# Transform the Dataframe
# Create a BigQuery client
# See if the target dimension table exists
#    If not exists, load the data into a new table
#    If exists, insert new records into the table
if __name__ == "__main__":
    df = pd.DataFrame
    # Load in the data file
    with open(file_source_path, 'r') as data:
            df = pd.read_csv(data)
        # Set all of the column names to lower case letters
    df = df.rename(columns=str.lower)


    #df = load_csv_data_file(file_source_path, "my_311_data_WaterQuality.csv", df)
    # Transform the data
    df = transform_data(df)
    # Create the BigQuery Client
    # setup enviroment parameters to connect to BQ project
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path_to_service_account_key_file

    # Construct a BigQuery client object
    bqclient = bigquery.Client()

    # See if the target dimensional table exists
    target_table_exists = bigquery_table_exists(bqclient, dimension_table_path  )

    # If the target dimension table does not exist, load all of the data into a new table
    if not target_table_exists:
        build_new_table( bqclient, dimension_table_path, dimension_name, df)
    # If the target table exists, then perform an incremental load



